# Regression tables with `ETable`

Regression tables are generated with `ETable()`. `ETable` inherits from the `MTable` base class, which provides all the core output functionality. This means that `ETable` can generate tables in multiple formats (HTML/GT, docx, LaTeX).

Let us load a sample dataset and run some regressions (here using [PyFixest](https://py-econometrics.github.io/pyfixest/pyfixest.html) but you can also use [statsmodels](https://www.statsmodels.org/stable/index.html)):

In [37]:
# Import necessary libraries
import numpy as np
import pandas as pd
import pyfixest as pf
import statsmodels.formula.api as smf
import maketables as mt

# Load sample dataset
df = pd.read_csv("data/salaries.csv")

# Estimate regressions for examples
est1 = pf.feols("logwage ~ age", data=df)
est2 = pf.feols("logwage ~ age + female", data=df)
est3 = pf.feols("logwage ~ age + female | worker_type", data=df)
est4 = pf.feols("logwage ~ age + female | worker_type + occupation", data=df)



## Basic Usage

The simplest way to create to simply call `ETable()` with a list of models:


In [2]:
mt.ETable([est1, est2, est3, est4])

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee0fcb4a0>

You can also estimate and display multiple regressions with one line of code using the (py)fixest stepwise notation:

In [3]:
mt.ETable(pf.feols("logwage+wage~csw(age, female, age:female)", data=df))

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee0e1bfb0>

## Keep and drop variables
For example, we can only keep the variables that we'd like, which keeps all variables that fit the provided regex match.


In [4]:
mt.ETable([est1, est2, est3, est4], keep="age")

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee7397c50>

We can use the `exact_match` argument to select a specific set of variables:

In [5]:
mt.ETable([est1, est2, est3, est4], keep=["age", "female"], exact_match=True)

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee5ecfcb0>

We can also easily **drop** variables via the `drop` argument:

In [6]:
mt.ETable([est1, est2, est3, est4], drop=["age"])

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee5ecfcb0>

## Specify displayed model statistics
The user can specify the model statistics to be displayed and their order by passing a list of strings to `model_stats`. Names of the statistics must match the model's respective attribute names such as "r2", "adj_r2", "N", "r2_within" (see the respective function reference for attributes and omit the leading "_"). The type of standard error estimated can be shown by adding "se_type" to the list. When passing an empty list no model statistics are displayed.

In [7]:
mt.ETable([est1, est2, est3, est4], model_stats=['N','r2','r2_within'])

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee738e6f0>

Model statistics can also be relabeled by passing a dictionary `model_stats_labels`.

In [8]:
mt.ETable([est1, est2, est3, est4], model_stats=['N','r2'], model_stats_labels={"N": "Number of observations", "r2": "R squared"})

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee738f590>

## Hide fixed effects
We can hide the rows showing the relevant fixed effects by setting `show_fe=False` (for instance when the set of fixed effects is the same for all models and you want to describe this in the text or table notes rather than displaying it in the table).

In [9]:
mt.ETable([est1, est2, est3, est4], model_stats=['r2','N'], show_fe=False)

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee7208920>

## Display p-values or confidence intervals
By default, `to.ETable()` reports **standard errors**. But we can also ask to output p-values or confidence intervals via the `coef_fmt` function argument.


In [10]:
mt.ETable([est1, est2, est3, est4], coef_fmt="b \n (se) \n [p]")

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee47be510>

## Formatting coefficients and standard errors
You can control how coefficients and standard errors are formatted using the `coef_fmt` parameter. This accepts Python format specifiers like ".3f" for 3 decimal places, or ".2e" for scientific notation.

In [11]:
# Example: Show coefficients and standard errors with 3 decimal places
mt.ETable([est1, est2, est3], coef_fmt="b:.3f \n (se:.3f)")

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee981f770>

## Rename variables
You can also rename variables if you want to have a more readable output. Just pass a dictionary to the `labels` argument. Note that interaction terms will also be relabeled using the specified labels for the interacted variables (if you want to manually relabel an interaction term differently, add it to the dictionary).


In [26]:
labels = {
    "logwage": "ln(Wage)",
    "wage": "Wage",
    "age": "Age",
    "female": "Female",
    "tenure": "Years of Tenure",
    "occupation": "Occupation",
    "worker_type": "Worker Type",
    "education": "Education Level"
}

mt.ETable([est1, est2, est3, est4], labels=labels)

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee720be30>

If you want to label the rows indicating the inclusion of fixed effects not with the variable label but with a custom label, you can pass on a separate dictionary to the `felabels` argument.


In [27]:
mt.ETable(
    [est1, est2, est3, est4],
    labels=labels,
    felabels={"occupation": "Occupation Fixed Effects", "worker_type": "Worker Type Fixed Effects"},
)

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee7397e00>

## Setting defaults
You can set default options for `ETable` by changing the class attributes. For example, to change the default model statistics, always use the same labels, digits, and significance levels, you can do the following:

In [32]:
# Set default options for all future ETable calls
mt.ETable.DEFAULT_MODEL_STATS = ["N","r2","r2_within"]
mt.ETable.DEFAULT_LABELS = labels
mt.ETable.DEFAULT_COEF_FMT = "b:.3f \n (se:.3f)"  # 3 decimal places for both coefficients and SEs
mt.ETable.DEFAULT_SIGNIF_CODE = [0.01, 0.05, 0.1]

In [38]:
mt.ETable([est1, est2, est3, est4])

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee4aa8110>

## Rename categorical variables

By default, categorical variables are returned using the formulaic "C(variable)[T.value]" notation. Via the `cat_template` argument,
you can rename categorical variables via a specified template *{variable}={value}*. This works when either the variable is categorial in the DataFrame, or the C() or i() operators are used in the regresson formula.
´

In [36]:
est5 = pf.feols("logwage ~ age + female + education", data = df)

mt.ETable([est5], labels=labels, cat_template = "{variable}: {value}")

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee0e1bfb0>

But you can also remove the variable name and only keep the levels (categories) by specifying *cat_template="{value}"*. Note that the labeling of categories also works in interaction terms:

In [19]:
est6 = pf.feols("logwage ~ age + female + education", data = df)
est7 = pf.feols("logwage ~ age + female + education*age", data = df)

mt.ETable([est6, est7], labels=labels, cat_template="{value}")

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee738df70>

## Custom model headlines
You can also add custom headers for each model by passing a list of strings to the `model_headers` argument.


In [39]:
mt.ETable(
    [est1, est2, est3, est4],
    labels=labels,
    model_heads=["OLS", "OLS", "Fixed Effects", "Fixed Effects"],
)

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee48ac830>

Or change the ordering of headlines having headlines first and then dependent variables using the `head_order` argument. "hd" stands for headlines then dependent variables, "dh" for dependent variables then headlines. Assigning "d" or "h" can be used to only show dependent variables or only headlines. When head_order="" only model numbers are shown.


In [50]:
mod1 = pf.feols("logwage ~ age + female", data=df)
mod2 = pf.feols("wage ~ age + female", data=df)
mod3 = pf.feols("logwage ~ age + female | worker_type + occupation", data=df)
mod4 = pf.feols("wage ~ age + female | worker_type + occupation", data=df)

mt.ETable([mod1, mod2, mod3, mod4], 
          model_heads=["OLS", "OLS", "Fixed Effects", "Fixed Effects"], 
          head_order="hd")
    

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee7097fe0>

Remove the dependent variables from the headers:

In [52]:
mt.ETable(
    [est1, est2, est3, est4],
    labels=labels,
    head_order="",
)

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee491c890>

## Further custom model information
You can add further custom model statistics/information to the bottom of the table by using the `custom_stats` argument to which you pass a dictionary with the name of the row and lists of values. The length of the lists must be equal to the number of models.


In [23]:
mt.ETable(
    [est1, est2, est3, est4],
    labels=labels,
    custom_model_stats={
        "Fixed Effects": ["No", "No", "Worker Type", "Worker Type + Occupation"],
        "Data Type": ["Cross-section", "Cross-section", "Cross-section", "Cross-section"],
    },
)

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2dee491f200>

## Custom table notes
You can replace the default table notes with your own notes using the `notes` argument.

In [55]:
mynotes = "This table shows wage regressions using a simulated dataset of 1,800 workers. The dependent variable is log wage. Models progressively add controls and fixed effects."
mt.ETable(
    [est1, est2, est3, est4],
    labels=labels,
    notes=mynotes,
)

<maketables.mtable.MTable.make.<locals>.DualOutput at 0x2deee09ab70>